## Libraries

In [3]:
import scipy.io
import matplotlib.pyplot as plt
import neurokit2 as nk
import pandas as pd
import os
import numpy as np

from utils import find_first, time_hour

In [7]:
# List all files in the "PASS_MUSE" directory
files = os.listdir("PASS_MUSE/")
print("Files in PASS_MUSE directory:", files)

# Load the corrected markers data
markers_corrected = pd.read_csv("correctedMarkers.csv")
print("First few rows of corrected markers data:")
print(markers_corrected.head())

# Get unique participants from the corrected markers data
part_un = markers_corrected["Participant"].unique()
print("Unique participants in corrected markers data:", part_un)

# Load the target data and drop rows with missing values
target = pd.read_csv("DATA_PASS.csv")
target = target.dropna()

# Get unique participants from the target data
part = target["Participant Number"].unique()
print("Unique participants in target data:", part)

# Find the intersection of participants in both datasets
part_markcorr = np.intersect1d(part, part_un)
print("Participants present in both datasets:", part_markcorr)

Files in PASS_MUSE directory: ['2137_1.mat', '2113_1.mat', '2111_1.mat', '2135_1.mat', '2108_1.mat', '2131_1.mat', '2115_1.mat', '2117_1.mat', '2131_2.mat', '2133_1.mat', '2110_1.mat', '2109_1.mat', '2134_1.mat', '2136_1.mat', '2112_1.mat', '2116_1.mat', '2132_1.mat', '2148_1.mat', '2130_1.mat', '2114_1.mat', '2129_1.mat', '2103_2.mat', '2125_1.mat', '2118_1.mat', '2144_1.mat', '2103_3.mat', '2146_1.mat', '2103_1.mat', '2144_3.mat', '2144_2.mat', '2127_1.mat', '2123_1.mat', '2142_1.mat', '2107_1.mat', '2138_1.mat', '2144_5.mat', '2140_1.mat', '2105_1.mat', '2121_1.mat', '2144_4.mat', '2124_2.mat', '2102_1.mat', '2147_1.mat', '2126_1.mat', '2143_4.mat', '2119_1.mat', '2124_1.mat', '2145_1.mat', '2104_1.mat', '2141_1.mat', '2143_3.mat', '2139_1.mat', '2143_2.mat', '2120_1.mat', '2104_2.mat', '2122_1.mat', '2106_1.mat', '2143_1.mat']
First few rows of corrected markers data:
             timestamp  Status  Participant
0  2017-03-03 09:20:59      10         2101
1  2017-03-03 09:23:26     

[2101 2102 2104 2105 2106 2107 2108 2109 2110 2111 2112 2113 2114 2115
 2116 2117 2118 2119 2120 2121 2122 2123 2124 2125 2126 2127 2129 2130
 2131 2132 2133 2134 2136]
[2104 2105 2106 2107 2108 2109 2110 2111 2112 2113 2114 2115 2116 2117
 2119 2120 2121 2122 2123 2124 2125 2126 2127 2129 2130 2131 2132 2133
 2134 2135 2136 2137 2317 2138 2139 2140 2141 2142 2145 2147 2148]
[2104 2105 2106 2107 2108 2109 2110 2111 2112 2113 2114 2115 2116 2117
 2119 2120 2121 2122 2123 2124 2125 2126 2127 2129 2130 2131 2132 2133
 2134 2136]
